In [ ]:
import os
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from numpy import transpose
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import parse_demand, generate_od_xy, add_centroids, get_centroid_grid_coords, auto_configured_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand, show_link_od_flows
from dyntapy.results import get_selected_link_analysis, get_od_flows

In [ ]:
# read in the network --> ???? DOESN'T WORK YET, i don't know how to read it in.
# Also, the netw object should be of type DiGraph 
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'
network_path = data_path + os.path.sep + "network_data" + os.path.sep + "BRUSSEL_3.osm"
print(network_path)
with open(network_path, 'r') as network_file:
    netw = network_file
    print(netw)
#show_network(netw, notebook=True)

In [ ]:
# read in the zoning file
zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + "BRUSSEL_3_10" + os.path.sep + "BRUSSEL_3_10.shp"
zoning = gpd.read_file(zoning_path)
print(zoning.head())
zone_numbers = zoning["ZONENUMMER"]
x_centroids = zoning["centroid_x"]
y_centroids = zoning["centroid_y"]

In [ ]:
# read in the od matrix 
od_path = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + "BRUSSEL_ext_tr_3_9_10.xlsx"
od = pd.read_excel(od_path)
od_array = od.to_numpy()
length = len(od_array)
origin_column = [i for i in zone_numbers for j in zone_numbers]
destination_column = [j for i in zone_numbers for j in zone_numbers]
flow_column = [od_array[i,j] for i in range(0,length) for j in range(0,length)]
origin_name = "origin"
destination_name = "destination"
flow_name = "flow"
od_frame = pd.DataFrame(transpose([origin_column, destination_column, flow_column]),columns=[origin_name, destination_name, flow_name])
print(od_frame.head())

In [ ]:
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids)

#Can't use show_network or show_demand, errors occur 
#show_demand(od_graph, notebook=True)
#show_network(od_graph, notebook=True)

# Volgens visualization.py 678-685 zou dit toch moeten werken? 

In [ ]:
# Once all 3 files are correctly laoded, we can run an assignment. 
tolls = np.zeros(netw.number_of_edges())
assignment = StaticAssignment(netw,od_graph, tolls)
methods = ['dial_b']
for method in methods:
    result = assignment.run(method)
    show_network(netw, flows = result.flows, notebook=True, show_nodes=False)
    print(f'{method=} ran successfully')
# all static assignments return a result object that follows the same structure, see
# below
print(result.__dict__.keys())